In [1]:
import glob
import pandas as pd

In [2]:
df = pd.concat([pd.read_parquet(x) for x in glob.glob('/raid/preprocess/*.parquet')])

In [3]:
df['hashtags'] = df['hashtags'].fillna('UNKNOWN')
df['links'] = df['links'].fillna('UNKNOWN')
df['domains'] = df['domains'].fillna('UNKNOWN')

In [7]:
df.head()

,timestamp,a_follower_count,a_following_count,a_account_creation,b_follower_count,b_following_count,b_account_creation,hashtags,tweet_id,media,...,b_is_verified,b_follows_a,reply,retweet,retweet_comment,like,hashtags_count_t,domains_count_t,links_count_t,media_splitmedia
0,1581378009,2730024,651,1295048494,80,322,1427577415,1,42322883,None,...,False,False,0,0,0,0,0,0,0,13
1,1581485160,744,1047,1528286720,780,1584,1436342848,1,12838860,Photo\tPhoto\tPhoto\tPhoto,...,False,False,0,0,0,0,0,0,0,7
2,1581512501,630,642,1533800339,280,253,1570192118,1,12056524,Photo,...,False,True,0,0,0,1581539236,0,0,0,5
3,1581048122,16235992,9,1221417992,250,234,1393788996,1,14683415,Photo,...,False,False,0,0,0,0,0,0,0,5
4,1581267159,88386,4994,1221280889,19,139,1500997563,1,12081839,None,...,False,False,0,0,0,1581269009,0,0,0,13


In [11]:
df['media'] = df['media_splitmedia'].values

In [12]:
df['dt_dow'] = pd.to_datetime(df['timestamp'], unit='s').dt.weekday

In [13]:
%%time
# TRAIN FIRST 5 DAYS. VALIDATE LAST 2 DAYS
VALID_DOW = [1, 2]# order is [3, 4, 5, 6, 0, 1, 2]
valid = df[df['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)
train = df[~df['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)

CPU times: user 19 s, sys: 14.3 s, total: 33.3 s
Wall time: 33.3 s


In [14]:
train = train.sort_values(["b_user_id", "timestamp"]).reset_index()
valid = valid.sort_values(["b_user_id", "timestamp"]).reset_index()

In [15]:
train.shape, valid.shape

((87914868, 29), (33471563, 29))

In [16]:
train.to_parquet('/raid/nv_train-1-train.parquet', row_group_size=1024)

In [17]:
valid.to_parquet('/raid/nv_train-1-valid.parquet', row_group_size=1024)